In [13]:
import pandas as pd
from pathlib import Path

# ==== 1) Okunacak Parquet dosyasını belirtin ====
parquet_file = Path("data/tefas_full_2yrs.parquet")   # burayı değiştirin

# ===============================================

# Parquet dosyasını oku
if not parquet_file.exists():
    raise FileNotFoundError(f"Parquet bulunamadı: {parquet_file}")

df = pd.read_parquet(parquet_file)
display(df.head())                 # ilk satırları göster
print(f"Satır: {len(df):,}  |  Sütun: {df.shape[1]}")

# ==== 2) Excel’e dönüştür ====
excel_file = parquet_file.with_suffix(".xlsx")
df.to_excel(excel_file, index=False)
print(f"Excel dosyası oluşturuldu: {excel_file}")

,tarih,fiyat,tedavuldeki_pay_sayisi,toplam_deger,yatirimci_sayisi,fon_unvan,fon_kodu,fon_kategorisi
0,2023-07-14,12.85490,6881972.0,88467064.59,535,ATA PORTFÖY ÇOKLU VARLIK DEĞİŞKEN FON,AAK,Değişken Şemsiye Fonu
1,2023-07-17,12.94562,6822709.0,88324200.73,540,ATA PORTFÖY ÇOKLU VARLIK DEĞİŞKEN FON,AAK,Değişken Şemsiye Fonu
2,2023-07-18,13.16363,6844889.0,90103583.75,545,ATA PORTFÖY ÇOKLU VARLIK DEĞİŞKEN FON,AAK,Değişken Şemsiye Fonu
3,2023-07-19,13.13001,6616862.0,86879466.07,554,ATA PORTFÖY ÇOKLU VARLIK DEĞİŞKEN FON,AAK,Değişken Şemsiye Fonu
4,2023-07-20,13.39776,6663617.0,89277539.59,556,ATA PORTFÖY ÇOKLU VARLIK DEĞİŞKEN FON,AAK,Değişken Şemsiye Fonu


Satır: 337,737  |  Sütun: 8
Excel dosyası oluşturuldu: data\tefas_full_2yrs.xlsx


In [2]:
# =======================================================================
# UNIQUE FON BİLGİLERİNİ ÇIKAR VE EXCEL'E BAS
# =======================================================================

# Gerekli sütunları kontrol et
required_cols = ['fon_kodu', 'fon_unvan', 'fon_kategorisi']
missing_cols = [col for col in required_cols if col not in df.columns]

if missing_cols:
    print(f"❌ Eksik sütunlar: {missing_cols}")
    print(f"📊 Mevcut sütunlar: {list(df.columns)}")
else:
    print("✅ Gerekli sütunlar mevcut!")
    
    # Unique fon bilgilerini al
    # Her fon için ilk kaydı alıyoruz (çünkü fon bilgileri aynı olmalı)
    unique_funds_df = df.groupby('fon_kodu').first().reset_index()[required_cols]
    
    print(f"📊 Unique fon sayısı: {len(unique_funds_df)}")
    
    # Kategori dağılımını göster
    print("\n🏷️ FON KATEGORİ DAĞILIMI:")
    category_series = unique_funds_df['fon_kategorisi']
    category_counts = category_series.value_counts()
    for category, count in category_counts.items():
        percentage = (count / len(unique_funds_df)) * 100
        print(f"   📈 {category}: {count:3d} fon ({percentage:5.1f}%)")
    
    # İlk 10 fonu göster
    print(f"\n📋 İLK 10 FON ÖRNEĞİ:")
    print(unique_funds_df.head(10).to_string(index=False))
    
    # Excel'e kaydet
    excel_file = Path("data/fon_kategorileri_kontrol.xlsx")
    unique_funds_df.to_excel(excel_file, index=False, sheet_name="Fon_Kategorileri")
    
    print(f"\n✅ Excel dosyası oluşturuldu: {excel_file}")
    print(f"📊 Toplam satır: {len(unique_funds_df)}")
    print(f"📊 Sütunlar: {list(unique_funds_df.columns)}")
    
    # Potansiyel problemleri kontrol et
    print(f"\n🔍 KALİTE KONTROLLERI:")
    
    # Boş kategori kontrolü
    empty_categories = category_series.isna().sum()
    print(f"   📋 Boş kategori: {empty_categories} fon")
    
    # 'Diğer' kategorisi kontrolü
    other_categories = category_series.str.contains('Diğer', case=False, na=False).sum()
    print(f"   📋 'Diğer' kategorisi: {other_categories} fon")
    
    # Çok kısa fon adları (potansiyel veri problemi)
    name_series = unique_funds_df['fon_unvan']
    short_names = (name_series.str.len() < 10).sum()
    print(f"   📋 Çok kısa fon adı (<10 karakter): {short_names} fon")
    
    if empty_categories > 0 or other_categories > 50 or short_names > 0:
        print(f"\n⚠️  Potansiyel veri kalite problemleri tespit edildi!")
    else:
        print(f"\n✅ Veri kalitesi iyi görünüyor!")


✅ Gerekli sütunlar mevcut!
📊 Unique fon sayısı: 850

🏷️ FON KATEGORİ DAĞILIMI:
   📈 Serbest Şemsiye Fonu: 281 fon ( 33.1%)
   📈 Değişken Şemsiye Fonu: 136 fon ( 16.0%)
   📈 Hisse Senedi Şemsiye Fonu: 128 fon ( 15.1%)
   📈 Katılım Şemsiye Fonu:  93 fon ( 10.9%)
   📈 Fon Sepeti Şemsiye Fonu:  73 fon (  8.6%)
   📈 Borçlanma Araçları Şemsiye Fonu:  50 fon (  5.9%)
   📈 Para Piyasası Şemsiye Fonu:  43 fon (  5.1%)
   📈 Kıymetli Madenler Şemsiye Fonu:  20 fon (  2.4%)
   📈 Yabancı Hisse Senedi Şemsiye Fonu:  11 fon (  1.3%)
   📈 Eurobond Şemsiye Fonu:   8 fon (  0.9%)
   📈 Diğer Şemsiye Fonu:   7 fon (  0.8%)

📋 İLK 10 FON ÖRNEĞİ:
fon_kodu                                                                                    fon_unvan             fon_kategorisi
     AAK                                                        ATA PORTFÖY ÇOKLU VARLIK DEĞİŞKEN FON      Değişken Şemsiye Fonu
     AAL                                                          ATA PORTFÖY PARA PİYASASI (TL) FONU Para Pi